### Best architecture

In [ ]:
# Use ImageDataGenerator to feed into the network in batch sizes
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.3,
    vertical_flip=True,
    horizontal_flip=True
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.3,
    vertical_flip=True,
    horizontal_flip=True
)

# Set paths and parameters
train_gen_path = MAIN_PATH / 'train_gen'
test_gen_path = MAIN_PATH / 'test_gen'
seed=100

# Set generation object path
train_gen = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(150, 150),
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=seed,
    save_to_dir=train_gen_path
)

test_gen = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(150, 150),
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=seed,
    save_to_dir=test_gen_path
)

# (1)

In [ ]:
# Define model parameters
EPOCHS = 10
BATCH_SIZE = 32

# Build the neural network
def build_model():
    
    # Input layer
    inputs = Input(shape=(150, 150, 3), name='input_layer')
    
    # First convolutional block
    x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same', name='conv1_1')(inputs)
    x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same', name='conv1_2')(x)
    x = MaxPool2D(pool_size=(2, 2), name='pool1_1')(x) # max pooling operation for spatial data
    
    # Second convolutional block
    x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='conv2_1')(x)
    x = BatchNormalization(name='bn2_1')(x) # stabilizing the learning process; reduce internal covariate shifting
    x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='conv2_2')(x)
    x = BatchNormalization(name='bn2_2')(x) 
    x = MaxPool2D(pool_size=(2, 2), name='pool2_1')(x)
    
    # Third convolutional block
    x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='conv3_1')(x)
    x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='conv3_2')(x)
    x = BatchNormalization(name='bn3_1')(x) 
    x = MaxPool2D(pool_size=(2, 2), name='pool3_1')(x)   
    
    # Fourth convolutional block
    x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='conv4_1')(x)
    x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='conv4_2')(x)
    x = BatchNormalization(name='bn4_1')(x) 
    x = MaxPool2D(pool_size=(2, 2), name='pool4_1')(x)
    x = Dropout(rate=0.20, name='dropout4_1')(x) # prevent network from overfitting; probabilistically reduce the network capacity
    
    # Fifth convolutional block
    x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', name='conv5_1')(x)
    x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', name='conv5_2')(x)
    x = BatchNormalization(name='bn5_1')(x) 
    x = MaxPool2D(pool_size=(2, 2), name='pool5_1')(x)
    x = Dropout(rate=0.30, name='dropout5_1')(x) 
    
    # Fully connected block
    x = Flatten(name='flatten6_1')(x)
    x = Dense(512, activation='relu', name='fc6_1')(x)
    x = Dropout(0.7, name='dropout6_1')(x)
    x = Dense(128, activation='relu', name='fc6_2')(x)
    x = Dropout(0.5, name='dropout6_2')(x)
    x = Dense(64, activation='relu', name='fc6_3')(x)
    x = Dropout(0.3, name='dropout6_3')(x)
    
    # Output
    output = Dense(units=1, activation='sigmoid', name='output')(x) # 1 so fewer parameters and computation are needed
    
    # Define model
    model = Model(inputs=inputs, outputs=output)
    return model

Epoch 1/10

163/163 [==============================] - 82s 502ms/step - loss: 0.3422 - acc: 0.8388 - val_loss: 1.7437 - val_acc: 0.6480

Epoch 2/10

163/163 [==============================] - 76s 464ms/step - loss: 0.2893 - acc: 0.8823 - val_loss: 0.5643 - val_acc: 0.6723

Epoch 3/10

163/163 [==============================] - 76s 465ms/step - loss: 0.2572 - acc: 0.8915 - val_loss: 0.7408 - val_acc: 0.5659

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.

Epoch 4/10

163/163 [==============================] - 76s 467ms/step - loss: 0.2121 - acc: 0.9185 - val_loss: 0.4917 - val_acc: 0.8598

Epoch 5/10

163/163 [==============================] - 76s 465ms/step - loss: 0.1995 - acc: 0.9247 - val_loss: 1.5835 - val_acc: 0.6385

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.

Epoch 6/10

163/163 [==============================] - 76s 469ms/step - loss: 0.1881 - acc: 0.9329 - val_loss: 0.3183 - val_acc: 0.9003

Epoch 7/10

163/163 [==============================] - 79s 487ms/step - loss: 0.1677 - acc: 0.9398 - val_loss: 0.3593 - val_acc: 0.8767

Epoch 00007: ReduceLROnPlateau reducing learning rate to 2.700000040931627e-05.

Epoch 8/10

163/163 [==============================] - 77s 471ms/step - loss: 0.1685 - acc: 0.9375 - val_loss: 0.3603 - val_acc: 0.8885

Epoch 9/10

163/163 [==============================] - 77s 471ms/step - loss: 0.1590 - acc: 0.9419 - val_loss: 0.5028 - val_acc: 0.8412

Epoch 00009: ReduceLROnPlateau reducing learning rate to 8.100000013655517e-06.

Epoch 10/10

163/163 [==============================] - 76s 469ms/step - loss: 0.1594 - acc: 0.9440 - val_loss: 0.3789 - val_acc: 0.8733

# (3)

In [ ]:
# # Build the neural network
# def build_model():
    
#     # Input layert
#     inputs = Input(shape=(TARGET_SIZE, TARGET_SIZE, 3), name ='input_layer')
    
#     # First convolutional block
#     x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same', name='conv1_1')(inputs)
#     x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same', name='conv1_2')(x)
#     x = MaxPool2D(pool_size=(2, 2), name='pool1_1')(x) # max pooling operation for spatial data
    
#     # Second convolutional block
#     x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='conv2_1')(inputs)
#     x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='conv2_2')(x)
#     x = BatchNormalization(name='bn2_1')(x) # stabilizing the learning process; reduce internal covariate shifting
#     x = MaxPool2D(pool_size=(2, 2), name='pool2_1')(x)
    
#     # Third convolutional block
#     x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='conv3_1')(x)
#     x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='conv3_2')(x)
#     x = BatchNormalization(name='bn3_1')(x) 
#     x = MaxPool2D(pool_size=(2, 2), name='pool3_1')(x)   
    
#     # Fourth convolutional block
#     x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='conv4_1')(x)
#     x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='conv4_2')(x)
#     x = BatchNormalization(name='bn4_1')(x) 
#     x = MaxPool2D(pool_size=(2, 2), name='pool4_1')(x)
#     x = Dropout(rate=0.20, name='dropout4_1')(x) # prevent network from overfitting; probabilistically reduce the network capacity
    
#     # Fifth convolutional block
#     x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='conv5_1')(x)
#     x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='conv5_2')(x)
#     x = BatchNormalization(name='bn5_1')(x) 
#     x = MaxPool2D(pool_size=(2, 2), name='pool5_1')(x)
#     x = Dropout(rate=0.20, name='dropout5_1')(x) 
    
#     # Fully connected block
#     x = Flatten(name='flatten6_1')(x)
#     x = Dropout(rate=0.5, name='dropout6_1')(x)
#     x = Dense(512, activation='relu', name='fc6_1')(x)

# #     x = Dropout(rate=0.7, name='dropout6_2')(x)
# #     x = Dense(128, activation='relu', name='fc6_2')(x)
# #     x = Dropout(rate=0.5, name='dropout6_3')(x)
# #     x = Dense(64, activation='relu', name='fc6_3')(x)
# #     x = Dropout(rate=0.3, name='dropout6_4')(x)
    
#     # Output
#     output = Dense(units=1, activation='sigmoid', name='output')(x) # 1 so fewer parameters and computation are needed
    
#     # Define model
#     model = Model(inputs=inputs, outputs=output)
#     return model

# (2)

In [ ]:
# # Build the neural network
# def build_model():
    
#     # Input layert
#     inputs = Input(shape=(TARGET_SIZE, TARGET_SIZE, 3), name ='input_layer')
    
#     # First convolutional block
#     x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='conv1_1')(inputs)
#     x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='conv1_2')(x)
#     x = MaxPool2D(pool_size=(2, 2), name='pool1_1')(x) # max pooling operation for spatial data
    
#     # Second convolutional block
#     x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='conv2_1')(x)
#     x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='conv2_2')(x)
#     x = BatchNormalization(name='bn2_1')(x) # stabilizing the learning process; reduce internal covariate shifting
#     x = MaxPool2D(pool_size=(2, 2), name='pool2_1')(x)
    
#     # Third convolutional block
#     x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='conv3_1')(x)
#     x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='conv3_2')(x)
#     x = BatchNormalization(name='bn3_1')(x) 
#     x = MaxPool2D(pool_size=(2, 2), name='pool3_1')(x)   
    
#     # Fourth convolutional block
#     x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', name='conv4_1')(x)
#     x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', name='conv4_2')(x)
#     x = BatchNormalization(name='bn4_1')(x) 
#     x = MaxPool2D(pool_size=(2, 2), name='pool4_1')(x)
#     x = Dropout(rate=0.20, name='dropout4_1')(x) # prevent network from overfitting; probabilistically reduce the network capacity
    
#     # Fifth convolutional block
#     x = SeparableConv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same', name='conv5_1')(x)
#     x = SeparableConv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same', name='conv5_2')(x)
#     x = BatchNormalization(name='bn5_1')(x) 
#     x = MaxPool2D(pool_size=(2, 2), name='pool5_1')(x)
#     x = Dropout(rate=0.20, name='dropout5_1')(x) 
    
#     # Fully connected block
#     x = Flatten(name='flatten6_1')(x)
#     x = Dense(512, activation='relu', name='fc6_1')(x)
#     x = Dropout(rate=0.5, name='dropout6_1')(x)
#     x = Dense(256, activation='relu', name='fc6_2')(x)
#     x = Dropout(rate=0.5, name='dropout6_2')(x)
# #     x = Dense(64, activation='relu', name='fc6_3')(x)
    
#     # Output
#     output = Dense(units=1, activation='sigmoid', name='output')(x) # 1 so fewer parameters and computation are needed
    
#     # Define model
#     model = Model(inputs=inputs, outputs=output)
#     return model